<a href="https://colab.research.google.com/github/ck1972/Hands-On-GeoAI1/blob/main/GeoAI_Lab_3b_Land_Cover_Classification_with_Machine_Learning_Classifiers_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Land Cover Classification Using Random Forest Classifier, Optical and SAR Imagery**

## Introduction
This lab will use a random forest classifier to classify optical (Sentinel-2) and SAR (ALOS PALSAR ScanSAR HV polarization) images. The goal is to improve land cover classification by combining optical and SAR imagery.

## Imports and Setup
### Install libraries
First, install any additional libraries that are not installed by default (e.g., rasterio, earthpy)..

In [ ]:
# Install rasterio and earthpy libraries
!pip install rasterio
!pip install earthpy

### Import libraries
Import the necessary libraries (pandas, numpy, scikit-learn, rasterio, etc.).

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import rasterio
import earthpy.plot as ep
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
import seaborn as sns
import joblib

### Mount Google Drive
Next, mount your Google Drive. You will be prompted to authorize access to your Google Drive. Once mounted, you can read/write files in /content/drive/MyDrive.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

### Define paths and variables
Define the the paths to access your own directory structure in Google Drive. In this tutorial, we use:
-A CSV training dataset (Bul_TrainingData_2024.csv) containing pixel values and their corresponding classes.
- A multiband Sentinel-2 image (Bul_S2_2024.tif).
- PALSAR ScanSAR polarization

In [ ]:
# Define path that contains the datasets
Sample_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2025/Bul_TA_S2_Pal_2025.csv'
S2_Image_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2025/Bul_S2_2025.tif'
Palsar_Image_Path = '/content/drive/MyDrive/Bulawayo_Dataset_2025/Bul_Palsar_HV_2025.tif'

### Define target and predictor variables
Next, define and specify the overall structure of the land cover classification task. Bands lists the Sentinel-2 spectral bands (e.g., B2, B3, B4) used as input features (predictors) for the model, while LC indicates the target column named “class.” Classes is a list of integer codes that the model will learn to predict, and N_Classes denotes the total number of these categories. The Names list provides descriptive labels (e.g., “Bare area,” “Built-up”) that match each code, making it easier to interpret results. Finally, Palette is a set of hex color codes for visualizing each class in plots or exported maps.

In [ ]:
# Define target and predictor variables
Bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'HV']  # Feature columns
LC = ['class']

Classes = [0, 1, 2, 3, 4, 5]
N_Classes = 6
Names   = ["Bare area", "Built-up", "Cropland", "Grassland", "Woodland", "Water"]
Palette = [
    '#D3D3D3',  # grey for class 0 (Bare area)
    '#FF0000',  # red for class 1 (Built-up)
    '#FFD700',  # gold for class 2 (Cropland)
    '#ADFF2F',  # greenyellow for class 3 (Grassland)
    '#006400',  # darkgreen for class 4 (Woodland)
    '#0000FF'   # blue for class 5 (Water)
]

## Load Images
We will use rasterio to open the .tif file. These are Sentinel-2 imagery and ALOS PALSAR ScanSAR HV polarization.

In [ ]:
# Load Sentinel-2 image
with rasterio.open(S2_Image_Path) as src_s2:
    s2_array = src_s2.read()  # shape: [bands, height, width]
    profile = src_s2.profile
    height, width = src_s2.height, src_s2.width

# Load PALSAR HV image
with rasterio.open(Palsar_Image_Path) as src_palsar:
    palsar_array = src_palsar.read(1)  # HV is single-band: [height, width]

# Ensure shapes match
assert s2_array.shape[1:] == palsar_array.shape, "Image sizes don't match. Reproject or resample needed."

### Display images
Display the Sentinel-2 composite and the PALSAR ScanSAR HV polorization image.

In [ ]:
# Select bands for RGB composite: B4 (Red), B3 (Green), B2 (Blue)
# Note: Check your band ordering if uncertain
red = s2_array[2, :, :]  # B4
green = s2_array[1, :, :]  # B3
blue = s2_array[0, :, :]  # B2

# Stack and normalize for RGB display
rgb = np.stack([red, green, blue], axis=-1)
rgb_min, rgb_max = 0, 0.3  # Adjust depending on your scaling
rgb_display = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)

# Normalized HV (grayscale display)
hv_min, hv_max = 0,1
hv_display = np.clip((palsar_array - hv_min) / (hv_max - hv_min), 0, 1)

# Plot side by side
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

axs[0].imshow(rgb_display)
axs[0].set_title('Sentinel-2 RGB (B4, B3, B2)')
axs[0].axis('off')

axs[1].imshow(hv_display, cmap='gray')
axs[1].set_title('PALSAR HV Backscatter')
axs[1].axis('off')

plt.tight_layout()
plt.show()

##  Load and Prepare Training Data
Next, load and prepare the training data. The training data is in a CSV format with columns for each band (B2, B3, etc.) and a class column (land cover type).

In [ ]:
# Load training data as a DataFrame
df = pd.read_csv(Sample_Path)

# Inspect first few rows
print(df.head())

# Separate features (X) and label (y)
X = df[Bands]
y = df['class']

# Ensure no missing values
print(f"Missing values in features: {X.isnull().sum().sum()}")
print(f"Missing values in label: {y.isnull().sum()}")

# Split into training and testing subsets
# (you can also do cross-validation if you prefer)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

## Exploratory Data Analysis (EDA)
### Summary statistics
Prepare summary statistics for the training dataset.

In [ ]:
# Descriptive statistics (mean, std, min, max, etc.)
summary_stats = X.describe().transpose()
summary_stats['missing_values'] = X.isnull().sum()
summary_stats['data_type'] = X.dtypes
print("Summary statistics for features:")
print(summary_stats)


### Check the class dsitribution
Next, check the land cover distribution in the training dataset.

In [ ]:
# Map class numbers to names
df['class_name'] = df['class'].map(dict(zip(Classes, Names)))

# Class distribution (absolute counts and percentages)
class_counts = df['class_name'].value_counts().sort_index()
class_percent = df['class_name'].value_counts(normalize=True).sort_index() * 100

# Combine into one table
class_summary = pd.DataFrame({
    'Class Name': class_counts.index,
    'Count': class_counts.values,
    'Percentage (%)': class_percent.values.round(2)
})

print("\nClass distribution:")
print(class_summary)

### Density plots
Density plots help visualize the distribution of pixel values in each spectral band for each land cover class.

In [ ]:
# Add readable class names
df['class_name'] = df['class'].map(dict(zip(Classes, Names)))

# Set up the 2x5 subplot grid for density plots
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
fig.suptitle('Density Plots by Land Cover Class', fontsize=20)

for i, band in enumerate(Bands):
    ax = axes[i // 5, i % 5]
    sns.kdeplot(
        data=df,
        x=band,
        hue="class_name",
        fill=True,
        common_norm=False,
        alpha=0.4,
        linewidth=1.2,
        ax=ax,
        palette=Palette
    )
    ax.set_title(f'Density Plot: {band}')
    ax.set_xlabel('Value')
    ax.set_ylabel('Density')

plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave space for suptitle
plt.show()

### Box plots

Box plots help you see the median, quartiles, and potential outliers across land cover classes.

In [ ]:
# Set up the 2x5 subplot grid for box plots
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
fig.suptitle('Box Plots by Land Cover Class', fontsize=20)

for i, band in enumerate(Bands):
    ax = axes[i // 5, i % 5]
    sns.boxplot(
        data=df,
        x="class_name",
        y=band,
        hue="class_name",           # <-- Add hue
        palette=dict(zip(Names, Palette)),
        legend=False,               # <-- Avoid legend on every plot
        ax=ax
    )
    ax.set_title(f'Box Plot: {band}')
    ax.set_xlabel('')
    ax.set_ylabel('Value')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Violin plots
 Violin plots combine box plots and kernel density estimates, giving you more insight into both distribution shape and summary statistics (median, quartiles, etc.)

In [ ]:
# Violin plots: 2 rows × 5 columns
fig, axes = plt.subplots(2, 5, figsize=(25, 10))
fig.suptitle('Violin Plots by Land Cover Class', fontsize=20)

for i, band in enumerate(Bands):
    ax = axes[i // 5, i % 5]
    sns.violinplot(
        data=df,
        x="class_name",
        y=band,
        hue="class_name",
        palette=dict(zip(Names, Palette)),
        dodge=False,
        ax=ax,
        linewidth=1
    )
    ax.set_title(f'Violin Plot: {band}')
    ax.set_xlabel('')
    ax.set_ylabel('Value')
    ax.tick_params(axis='x', rotation=45)

    # Safe legend removal
    if ax.get_legend() is not None:
        ax.get_legend().remove()

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

#### *Notes*
The width of the violin at any given point reflects the density of data points (or pixels, in the case of remote sensing). A wider section indicates that more observations fall within that range. A narrow neck in the violin shape suggests that the values are highly concentrated with low variation, while a wide base typically points to a long tail or spread in the data.

At the center of the violin, you'll find a box plot representation. The white dot indicates the median value, providing a quick view of the central tendency of the distribution. The thick black bar represents the interquartile range (IQR), the middle 50% of the data, while the thin vertical line extending above and below the box shows the data's range, excluding potential outliers.

The height of the violin itself shows the range of values a feature (or band) takes for a particular land cover class. A taller violin implies greater variability in that band for the class, while a shorter violin reflects more consistent or tightly clustered values. Collectively, these components make violin plots especially useful for assessing class separability, variability, and feature usefulness in classification tasks.

### Correlation matrix

This helps you understand the relationships among bands and HV.

In [ ]:
# Compute correlation matrix
corr = df[Bands].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', square=True)
plt.title('Feature Correlation Matrix')
plt.tight_layout()
plt.show()

Print correlations above 0.9.

In [ ]:
# Compute correlation matrix
corr = df[Bands].corr()

# Select upper triangle of the correlation matrix (excluding diagonal)
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Find and print pairs with correlation above 0.9
high_corr = upper.stack()[upper.stack() > 0.9]

print("Feature pairs with correlation > 0.9:")
print(high_corr)

## Train and Evaluate the Random Forest Classifier

We will train the random forest and will compile accuracy metrics, confusion matrices, and classification reports.

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

### Model validation
Next, we will calculate model validation metrics for the random forest model.

In [ ]:
# Evaluate each model
models = {
    "Random Forest": rf_preds
}

for name, preds in models.items():
    print(f"\n** {name} **")
    print("Accuracy:", (preds == y_test).mean())

    # Classification Report
    print(classification_report(y_test, preds))

    # Confusion Matrix
    cm = confusion_matrix(y_test, preds, labels=Classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=Classes)
    disp.plot(cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

## Save the Best RF Model
Let's save the best RF model parameters, feature names, or other metadata together.


In [ ]:
# Save the best model parameters
model_package = {
    "model": rf,            # your trained Random Forest model
    "features": Bands,      # input feature names
    "label": 'class'        # label column name
}

# Define model path
MODEL_PATH = '/content/drive/MyDrive/Bulawayo_Dataset_2025/best_rf_model.pkl'
joblib.dump(model_package, MODEL_PATH)
print("Model saved successfully.")

Model saved successfully.


## Land Cover Classification

Next, we will use the random forest classifier to predict class labels for every pixel of the Sentinel-2 and ALOS PALSAR images.

### i. Stack Sentinel-2 and PALSAR HV images

In [ ]:
# Stack PALSAR HV as an extra band
palsar_array_expanded = palsar_array[np.newaxis, :, :]  # shape: [1, height, width]
combined_array = np.concatenate((s2_array, palsar_array_expanded), axis=0)  # shape: [bands+1, height, width]
print("Combined array shape:", combined_array.shape)

### ii. Reshape for prediction
Scikit-Learn expects a 2D array for prediction, where each row corresponds to a pixel and each column corresponds to a band. Therefore, we need to reshape from (bands, height, width) to (height*width, bands) for input into Scikit-Learn.

In [ ]:
# Reshape for classification
band_count = combined_array.shape[0]
img_reshaped = combined_array.reshape(band_count, -1).T  # shape: [n_pixels, n_features]
print("Reshaped array for prediction:", img_reshaped.shape)

Reshaped array for prediction: (23761673, 10)


### iii.  Use the random forest for prediction
Next, we will use the random forest for prediction. Finally, we will take a 1D array of predicted land cover classes and reshape it back into a 2D raster format (image) so it matches the original image's spatial dimensions.

In [ ]:
# Predict using trained RandomForest model (e.g. rf)
prediction = rf.predict(img_reshaped)
prediction = prediction.astype(np.uint8)

# Reshape back to image
prediction_map = prediction.reshape(height, width)

### Visualize the land cover map

Next, visualize the resulting map with a color palette corresponding to each class ID.

In [ ]:
# Prepare a discrete colormap
# We need one more level than classes for from_levels_and_colors
levels = Classes + [max(Classes) + 1]
cmap, norm = from_levels_and_colors(levels, Palette)

plt.figure(figsize=(10,8))
im = plt.imshow(prediction_map, cmap=cmap, norm=norm)
plt.title("Land Cover Map")

# Create colorbar with a shrink factor (0.7 = 70% of default height)
cbar = plt.colorbar(im, shrink=0.7)

# Center tick labels
tick_positions = [i + 0.5 for i in Classes]
cbar.set_ticks(tick_positions)
cbar.set_ticklabels(Names)

plt.show()

## Export the Land Cover Map
We can save the land cover map to a new GeoTIFF using rasterio.

In [ ]:
# Save predicted land cover map
output_path = '/content/drive/MyDrive/Bulawayo_Dataset_2025/Bul_LC_RF_2025a.tif'
with rasterio.open(
    output_path, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=np.uint8,
    crs=profile['crs'],
    transform=profile['transform']
) as dst:
    dst.write(prediction_map, 1)